In [4]:
from pathlib import Path
import utils.sonar_analysis as iau

# Centralized sonar defaults (inserted by sweep)
from utils.config import SONAR_VIS_DEFAULTS
config = SONAR_VIS_DEFAULTS.copy()
# Backwards-compatible variable names used in older notebooks
RANGE_MIN_M = config['range_min_m']
RANGE_MAX_M = config['range_max_m']
DISPLAY_RANGE_MAX_M = config['display_range_max_m']

# Simple Image Analysis with CV2

This notebook demonstrates:
1. **Pick a frame** from NPZ files and save it locally
2. **Use standard cv2 functions** directly for image processing
3. **Experiment** with different OpenCV operations

In [ ]:
# Prefer selecting by bag ID (part of the NPZ filename) instead of a numeric index.
# Set TARGET_BAG to a substring that appears in the NPZ filename 
# '2024-08-22_14-47-39' #1
# '2024-08-22_14-29-05' #2
# '2024-08-20_14-22-12' #3
# '2024-08-20_14-31-29' #4
# '2024-08-20_18-47-40' #5
# '2024-08-20_13-55-34' #6
# '2024-08-20_13-57-42' #7
# '2024-08-20_17-02-00' #8
# '2024-08-20_14-57-38' #9
TARGET_BAG = '2024-08-20_14-57-38'  # change this to your desired bag ID
from utils.config import EXPORTS_DIR_DEFAULT, EXPORTS_SUBDIRS
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)

# If you keep your raw .bag files or raw data on an external drive, point DATA_DIR there.
# Example (your external disk): /Volumes/LaCie/SOLAQUA/raw_data
DATA_DIR = Path("/Volumes/LaCie/SOLAQUA/raw_data")
print(f"Using DATA_DIR = {DATA_DIR}")

# NOTE: To export data for this specific bag, use:
# python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem {TARGET_BAG} --all

# Find NPZ files and pick the one matching TARGET_BAG
files = iau.get_available_npz_files()
if not files:
    raise FileNotFoundError(f"No NPZ files found in configured exports outputs (looked under {EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('outputs','outputs')})")
matches = [p for p in files if TARGET_BAG in p.name]
if not matches:
    # Help the user by listing available NPZ files
    print(f'No NPZ file matched TARGET_BAG={TARGET_BAG!r}')
    print('Available NPZ files:')
    for i,p in enumerate(files):
        print(f'  {i}: {p.name}')
    raise ValueError(f'No NPZ file contains "{TARGET_BAG}" in its name')

# If multiple matches, choose the most recently modified one
selected = max(matches, key=lambda p: p.stat().st_mtime)
NPZ_FILE_INDEX = files.index(selected)

Using DATA_DIR = /Volumes/LaCie/SOLAQUA/raw_data


## Distance Analysis Over Time

Now let's perform a comprehensive analysis of the red line distance over time. The red line represents the major axis of the detected elongated contour (likely a fishing net), and we'll track how this distance changes throughout the video sequence.

In [ ]:
# Import video generation module
import utils.video_generation as sg
import importlib

# Force reload to pick up config changes
importlib.reload(sg)

# Verify the config value is correct
from utils.config import TRACKING_CONFIG as CHECK_CONFIG
print(f"Verified expansion factor: {CHECK_CONFIG['ellipse_expansion_factor']}")

print("GENERATING CONTOUR DETECTION PIPELINE VIDEO")
print("=" * 60)

# Generate contour detection video showing the exact analysis pipeline
try:
    video_path = sg.create_enhanced_contour_detection_video(
        npz_file_index=NPZ_FILE_INDEX,          
        frame_start=1,
        frame_count= 2000,
        frame_step=1,
        output_path=str(Path(EXPORTS_DIR_DEFAULT) / EXPORTS_SUBDIRS.get('videos','videos') / f"contour_detection_{TARGET_BAG}.mp4")
    )
    
except Exception as e:
    print(f"❌ Video generation failed: {e}")
    import traceback
    traceback.print_exc()

Verified expansion factor: 0.5
GENERATING CONTOUR DETECTION PIPELINE VIDEO
=== CONTOUR DETECTION PIPELINE VIDEO (3x3 Grid with NetTracker) ===
Processing 1473 frames...
Grid layout (2x3):
  Row 1: Raw | Momentum-Merged | Edges
  Row 2: Search Mask | Best Contour | Distance
Output grid size: 2700x1400
Tracker config:
  expansion: 0.5
  center_alpha: 0.8
  size_alpha: 0.01
  angle_alpha: 0.1
Processing 1473 frames...
Grid layout (2x3):
  Row 1: Raw | Momentum-Merged | Edges
  Row 2: Search Mask | Best Contour | Distance
Output grid size: 2700x1400
Tracker config:
  expansion: 0.5
  center_alpha: 0.8
  size_alpha: 0.01
  angle_alpha: 0.1
Processed 50/1473 frames
Processed 50/1473 frames
Processed 100/1473 frames
Processed 100/1473 frames
Processed 150/1473 frames
Processed 150/1473 frames
Processed 200/1473 frames
Processed 200/1473 frames
Processed 250/1473 frames
Processed 250/1473 frames
Processed 300/1473 frames
Processed 300/1473 frames
Processed 350/1473 frames
Processed 350/1473 fr

In [5]:
# Analyze distance over time using functional API
print("\n" + "=" * 60)
print("ANALYZING DISTANCE OVER TIME")
print("=" * 60)

net_analysis_results = iau.analyze_npz_sequence(
    npz_file_index=NPZ_FILE_INDEX,    
    frame_start=1,
    frame_count=1500,
    frame_step=1,
    save_outputs=True
)


ANALYZING DISTANCE OVER TIME
Analyzing 1473 frames from 2024-08-20_14-57-38_data_cones.npz
Using NetTracker system with binary processing and ellipse fitting
Analyzing 1473 frames from 2024-08-20_14-57-38_data_cones.npz
Using NetTracker system with binary processing and ellipse fitting
  50/1473 | Status: TRACKED
  50/1473 | Status: TRACKED
  100/1473 | Status: TRACKED
  100/1473 | Status: TRACKED
  150/1473 | Status: TRACKED
  150/1473 | Status: TRACKED
  200/1473 | Status: TRACKED
  200/1473 | Status: TRACKED
  250/1473 | Status: TRACKED
  250/1473 | Status: TRACKED
  300/1473 | Status: TRACKED
  300/1473 | Status: TRACKED
  350/1473 | Status: TRACKED
  350/1473 | Status: TRACKED
  400/1473 | Status: TRACKED
  400/1473 | Status: TRACKED
  450/1473 | Status: TRACKED
  450/1473 | Status: TRACKED
  500/1473 | Status: TRACKED
  500/1473 | Status: TRACKED
  550/1473 | Status: TRACKED
  550/1473 | Status: TRACKED
  600/1473 | Status: TRACKED
  600/1473 | Status: TRACKED
  650/1473 | Statu

In [6]:
net_analysis_results.columns

Index(['frame_index', 'timestamp', 'distance_pixels', 'distance_meters',
       'angle_degrees', 'detection_success', 'tracking_status', 'area'],
      dtype='object')

## Convert to Real-World Distances

Now let's convert the pixel distances to real-world distances using the fact that the entire sonar image represents a 10x10 meter area.

In [7]:
# Auto-detect pixel->meter mapping from the selected NPZ using utils function
from utils.sonar_utils import get_pixel_to_meter_mapping
mapping_info = get_pixel_to_meter_mapping(selected)

# Extract commonly used variables for backwards compatibility
pixels_to_meters_avg = mapping_info['pixels_to_meters_avg']
image_shape = mapping_info['image_shape'] 
sonar_coverage_meters = mapping_info['sonar_coverage_meters']

print(f"Using pixels_to_meters_avg = {pixels_to_meters_avg:.6f} m/px")
print(f"Mapping source: {mapping_info['source']}")

Using pixels_to_meters_avg = 0.016765 m/px
Mapping source: npz_metadata


In [8]:
from utils.sonar_visualization import plot_distance_analysis
plot_distance_analysis(net_analysis_results, "Real-World Distance Analysis")

In [9]:
# COMPARISON: SONAR vs DVL DISTANCE MEASUREMENTS
# =================================================
import utils.distance_measurement as sda
from utils.sonar_visualization import compare_sonar_vs_dvl

# IMPORTANT: Pass the by_bag folder, not just the exports root
# The function expects the folder containing the CSV files
from utils.config import EXPORTS_SUBDIRS
BY_BAG_FOLDER = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('by_bag', 'by_bag')

# Load all distance data for the target bag
raw_data, distance_measurements = sda.load_all_distance_data_for_bag(TARGET_BAG, BY_BAG_FOLDER)

# Display what we loaded
print(f"\nRAW DATA LOADED:")
for key, data in raw_data.items():
    if data is not None:
        print(f"    {key}: {len(data)} records")
    else:
        print(f"    {key}: None")

print(f"\nDISTANCE MEASUREMENTS LOADED:")
for key, info in distance_measurements.items():
    data_len = len(info['data'])
    print(f"    {key}: {data_len} records - {info['description']}")

 LOADING ALL DISTANCE DATA FOR BAG: 2024-08-20_14-57-38
 1. Loading Navigation Data...
    Loaded 814 navigation records
    NetPitch data available: 814 valid records
 2. Loading Guidance Data...
    Loaded 817 guidance records with ['error_net_distance', 'desired_net_distance', 'r_net_distance_d']
 3. Loading DVL Altimeter...
    DVL altimeter file not found
 4. Loading USBL...
    Loaded 47 USBL records
 5. Loading DVL Position...
    Loaded 414 DVL position records
 6. Loading Navigation Position...
    Loaded 412 navigation position records
 7. Loading INS Z Position...
    INS file not found

 LOADING SUMMARY:
    Target bag: 2024-08-20_14-57-38
    Raw data loaded: 2/2
    Distance measurements: 4

RAW DATA LOADED:
    navigation: 814 records
    guidance: 817 records

DISTANCE MEASUREMENTS LOADED:
    USBL_3D: 47 records - 3D acoustic position
    USBL_Depth: 47 records - USBL depth measurement
    DVL_Position: 414 records - 3D DVL position
    Nav_Position: 412 records - 2D n

In [10]:
# Compare sonar vs DVL using the sonar_visualization function
print(f"\n" + "=" * 60)
print("SONAR vs DVL COMPARISON")
print("=" * 60)

fig, comparison_stats = compare_sonar_vs_dvl(
    net_analysis_results,
    raw_data,
    sonar_coverage_m=sonar_coverage_meters,
    sonar_image_size=image_shape[0]
)

if fig:
    fig.show()

print(f"\nComparison Statistics:")
for key, value in comparison_stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.3f}")
    else:
        print(f"  {key}: {value}")


SONAR vs DVL COMPARISON

Comparison: Sonar=1.487m, DVL=1.508m, Ratio=0.986x



Comparison Statistics:
  sonar_mean_m: 1.487
  dvl_mean_m: 1.508
  scale_ratio: 0.986
  sonar_frames: 1473
  dvl_records: 814


In [9]:
# CONFIGURATION
FFT_CSV_PATH = Path("/Volumes/LaCie/SOLAQUA/relative_fft_pose")
TARGET_FFT_FILE = FFT_CSV_PATH / f"{TARGET_BAG}_relative_pose_fft.csv"

print("GENERATING THREE-SYSTEM VIDEO")
print("=" * 60)
print(f"Target Bag: {TARGET_BAG}")

# Import the utility function
from utils.video_generation import generate_three_system_video

try:
    # Generate video - automatically loads saved analysis CSV and DVL data
    video_path = generate_three_system_video(
        target_bag=TARGET_BAG,
        exports_folder=EXPORTS_FOLDER,
        fft_csv_path=None,  # Set to TARGET_FFT_FILE to enable FFT overlay
        start_idx=1,
        end_idx=1500  
    )
    
    if video_path:
        print(f"\n✓ Video generated successfully: {video_path}")
        import os
        if os.path.exists(video_path):
            size_mb = os.path.getsize(video_path) / (1024**2)
            print(f"  File size: {size_mb:.2f} MB")
    
except FileNotFoundError as e:
    print(f"\n❌ Missing required data files:")
    print(f"  {e}")
    print(f"\nMake sure you've run the sonar analysis cell above with save_outputs=True")
    
except Exception as e:
    print(f"\n❌ Video generation failed: {e}")
    import traceback
    traceback.print_exc()

GENERATING THREE-SYSTEM VIDEO
Target Bag: 2024-08-20_14-57-38
GENERATING TWO-SYSTEM VIDEO
📊 Loading sonar analysis results from saved CSV...
✓ Loaded 1473 sonar analysis records from 2024-08-20_14-57-38_data_cones_analysis.csv
📊 Loading DVL navigation data...
 LOADING ALL DISTANCE DATA FOR BAG: 2024-08-20_14-57-38
 1. Loading Navigation Data...
    Loaded 814 navigation records
    NetPitch data available: 814 valid records
 2. Loading Guidance Data...
    Loaded 817 guidance records with ['error_net_distance', 'desired_net_distance', 'r_net_distance_d']
 3. Loading DVL Altimeter...
    DVL altimeter file not found
 4. Loading USBL...
    Loaded 47 USBL records
 5. Loading DVL Position...
    Loaded 414 DVL position records
 6. Loading Navigation Position...
    Loaded 412 navigation position records
 7. Loading INS Z Position...
    INS file not found

 LOADING SUMMARY:
    Target bag: 2024-08-20_14-57-38
    Raw data loaded: 2/2
    Distance measurements: 4
✓ Loaded 814 DVL navigatio